In [2]:
import pandas as pd
import numpy as np

In [84]:
metadata = pd.read_csv("../data/03_ICIM_analysis/KC_ICIM_TSNE_data.csv")

In [85]:
metadata.head()

,CellID,Age,Gender,Genotype,Replicate,nGene,nUMI,cell_type_id,is_kc,x,y,hdb_clust,subtype
0,ACATACGAGGGCTTCC-DGRP-551_0d_r1,0,Female,DGRP-551,DGRP-551_0d_Rep1,1328,3340.0,8.0,1,-16.732498,55.850990,3,G-KC
1,ACCCACTTCACTCTTA-DGRP-551_0d_r1,0,Female,DGRP-551,DGRP-551_0d_Rep1,1613,4580.0,8.0,1,-19.377689,40.539936,3,G-KC
2,ACCGTAAAGATAGTCA-DGRP-551_0d_r1,0,Male,DGRP-551,DGRP-551_0d_Rep1,1466,4349.0,22.0,1,-1.656786,-67.122963,2,a/b-KC
3,ACTTACTAGTGGTAAT-DGRP-551_0d_r1,0,Male,DGRP-551,DGRP-551_0d_Rep1,1174,2942.0,8.0,1,-15.413997,46.957966,3,G-KC
4,ACTTGTTCATGGTTGT-DGRP-551_0d_r1,0,Male,DGRP-551,DGRP-551_0d_Rep1,1410,3620.0,8.0,1,-19.743160,40.827847,3,G-KC


In [86]:
metadata['dna_index'] = metadata.CellID.str.split('-').str[0]

In [87]:
metadata = metadata.drop(['nGene', 'nUMI', 'cell_type_id', 'is_kc', 'x', 'y', 'CellID', 'Replicate'], axis=1)
metadata = metadata.set_index('dna_index')

In [89]:
metadata.head()

,Age,Gender,Genotype,hdb_clust,subtype
dna_index,,,,,
ACATACGAGGGCTTCC,0,Female,DGRP-551,3,G-KC
ACCCACTTCACTCTTA,0,Female,DGRP-551,3,G-KC
ACCGTAAAGATAGTCA,0,Male,DGRP-551,2,a/b-KC
ACTTACTAGTGGTAAT,0,Male,DGRP-551,3,G-KC
ACTTGTTCATGGTTGT,0,Male,DGRP-551,3,G-KC


In [92]:
cell_number_info = pd.read_csv("../data/06_tss_data/indexed_barcodes.tsv", sep='\t', header=None)
cell_number_info.columns = ['cell_number', 'dna_index']
cell_number_info = cell_number_info.set_index("dna_index")

In [117]:
cell_number_info.columns = ['cell_number', 'dna_index']

In [94]:
metadata = metadata.join(cell_number_info)

In [96]:
metadata = metadata.set_index("cell_number")

In [10]:
metadata.head()

In [98]:
metadata.to_csv("../data/06_tss_data/umi_with_metadata.csv")

In [106]:
# Fix up UMI table by replacing flybase contigs with gene symbols
umi = pd.read_csv("../data/06_tss_data/all_umis_merged.tsv", sep="\t")
umi = umi.set_index('cell_number')

In [107]:
umi.head()

,contig,coord,back_coord,strand,numdup
cell_number,,,,,
682,FBgn0001222,142,171,-1,1
682,FBgn0030435,339,368,-1,1
682,FBgn0030435,393,422,-1,3
682,FBgn0261885,420,449,-1,1
682,FBgn0261885,434,463,-1,1


In [108]:
umi['facet'] = "Read Mappings"
umi = umi.drop(["strand", "numdup", "back_coord"], axis=1)

In [109]:
umi.head()

,contig,coord,facet
cell_number,,,
682,FBgn0001222,142,Read Mappings
682,FBgn0030435,339,Read Mappings
682,FBgn0030435,393,Read Mappings
682,FBgn0261885,420,Read Mappings
682,FBgn0261885,434,Read Mappings


In [71]:
## Using this list of genes to convert to gene symbols
## done with this online tool, here 
## http://flybase.org/convert/id
## downloaded to ../data/06_tss_data/FlyBase_IDs.txt
with open("../data/06_tss_data/flybase_gene_id_list_to_convert.txt", 'w') as f:
    for fb in umi.contig.unique():
        f.write("{}\n".format(fb))

In [110]:
# generate map df
converter = pd.read_csv("../data/06_tss_data/FlyBase_IDs.txt", sep="\t")
converter = converter[['submitted_id', 'current_symbol']]
converter = converter.set_index('submitted_id')

In [111]:
# Make sure every gene is accounted for
umi[~umi.contig.isin(converter.index)]

,contig,coord,facet
cell_number,,,


In [112]:
# We're good!
converter.columns = ['symbol']

In [113]:
umi = umi.join(converter, on='contig')

In [114]:
umi = umi.drop("contig", axis=1)

In [115]:
umi.head()

,coord,facet,symbol
cell_number,,,
682,142,Read Mappings,Hsf
682,339,Read Mappings,CG4645
682,393,Read Mappings,CG4645
682,420,Read Mappings,osa
682,434,Read Mappings,osa


In [116]:
umi.to_csv("../data/06_tss_data/all_umis_merged.csv")

In [67]:
## Fix up exon table as well
exon = pd.read_csv("../data/06_tss_data/exon_local_coords.tsv", sep='\t')
exon = exon.set_index("gene_id")
exon.head()

,exon,local_lesser_coord,local_greater_coord
gene_id,,,
FBgn0031081,4,2659,2775
FBgn0031081,6,3710,3830
FBgn0031081,7,3901,4215
FBgn0031081,8,4281,4775
FBgn0031081,9,4887,5716


In [68]:
exon['coord'] = exon.local_lesser_coord
exon["back_coord"] = exon.local_greater_coord
exon = exon.drop(["exon", "local_lesser_coord", "local_greater_coord"], axis=1)
exon['facet'] = "Exon Map"
exon.head()

,coord,back_coord,facet
gene_id,,,
FBgn0031081,2659,2775,Exon Map
FBgn0031081,3710,3830,Exon Map
FBgn0031081,3901,4215,Exon Map
FBgn0031081,4281,4775,Exon Map
FBgn0031081,4887,5716,Exon Map


In [109]:
# Need a new gene list for exon genes
## done with this online tool, here 
## http://flybase.org/convert/id
## downloaded to ../data/06_tss_data/exon_FlyBase_IDs.txt
with open("../data/06_tss_data/flybase_exon_gene_id_list_to_convert.txt", 'w') as f:
    for fb in exon.index.unique():
        f.write("{}\n".format(fb))

In [69]:
# generate map df
exon_converter = pd.read_csv("../data/06_tss_data/exon_FlyBase_IDs.txt", sep="\t")
exon_converter = exon_converter[['submitted_id', 'current_symbol']]
exon_converter = exon_converter.set_index('submitted_id')

In [70]:
# Make sure every gene is accounted for
exon[~exon.index.isin(exon_converter.index)]

,coord,back_coord,facet
gene_id,,,


In [71]:
# We're good!
exon_converter.columns = ['symbol']

In [72]:
exon = exon.join(exon_converter)
exon.index.name ='contig'
exon = exon.set_index("symbol")
exon = exon[exon.index.isin(umi.symbol.unique())]

In [104]:
exon.to_csv("../data/06_tss_data/exon_local_coords.csv")